## Attention

複数のデータの中から重要なデータに注目する仕組み。  
attentionを用いることで、decoderが、encoderが出力した情報の中の重要な情報に注目するようになる。

前章で作成した翻訳モデルは、入力文をencoderによって固定長のベクトルに変換し、それをdecoderに渡すことで、入力文に基づいた出力文を生成した。  
encoderがRNNであるとき、encoderは各時間で隠れ状態を出力する。この中から最後の時間の隠れ状態のみをdecoderに渡していたものがこれまでのseq2seqである。

このとき、encoderが出力する全ての隠れ状態を利用したいと考える。その方が入力文の多くの情報を参照でき、より適切な出力が得られそうだ。attentionはそれを実現する。

In [1]:
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(with_test=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 学習データの用意

In [3]:
textfile_ja = 'data/iwslt2017_ja_10000.txt'
textfile_en = 'data/iwslt2017_en_10000.txt'
tokenizer_prefix_ja = f'models/tokenizer_iwslt2017_ja_10000'
tokenizer_prefix_en = f'models/tokenizer_iwslt2017_en_10000'

with open(textfile_en) as f:
    data_en = f.read().splitlines()

with open(textfile_ja) as f:
    data_ja = f.read().splitlines()

n_data = len(data_en)
print('num of data:', n_data)

sp_ja = spm.SentencePieceProcessor(f'{tokenizer_prefix_ja}.model')
sp_en = spm.SentencePieceProcessor(f'{tokenizer_prefix_en}.model')
unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
pad_id = sp_ja.pad_id()
n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print('num of vocabrary (ja):', n_vocab_ja)
print('num of vocabrary (en):', n_vocab_en)

num of data: 10000
num of vocabrary (ja): 8000
num of vocabrary (en): 8000


In [4]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_ja.append(eos_id)
    ids_en.insert(0, bos_id)
    ids_en.append(eos_id)

In [5]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja
        x_dec = en[:-1]
        y_dec = en[1:]
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch):
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)


---

## Attention

attention機構について詳しく見ていく。

ある1つの入力と、関連する複数のデータを考える。複数のデータはmemoryと呼ぶ。  
入力を元に、memoryの中のどのデータに着目するかを定めることがattentionの目的である。各データに重要度を割り当てるという感じ。

重要度は重みと呼ばれ、$w_i$で表すことにする。  
重みは総和が1になるようにsoftmaxなどで正規化する。正規化前の値はスコアと呼んだりする。

各重みは入力との内積で求める。別に内積じゃなくてもいいけど、内積が一番簡単だし性能も良い。  
内積が取れるように、memoryの各ベクトルは入力と同じ次元にする必要がある。

重みを求めた後は、その重みでmemoryの重み付き和をとる。そうすることで、memoryの中から重要な要素を多めに取り出した固定長のベクトルが得られる。

やってみよう。  
入力と、3つのデータからなるmemoryを用意する。

In [6]:
n, d = 3, 5

x = torch.randn(d) # 入力
memory = torch.randn(n, d) # memory
memory

tensor([[-0.2305,  0.4712,  1.5261, -0.8287,  1.9600],
        [-1.3293, -1.4682,  1.5644,  0.0796, -1.7792],
        [-0.6182,  0.9205, -0.0433,  0.8591, -0.6128]])

入力とmemory内の全てのデータで内積を取り、softmaxで正規化する。

In [12]:
scores = memory @ x # スコア (内積)
weights = F.softmax(scores, dim=-1) # 重み
weights

tensor([0.6799, 0.1905, 0.1297])

スコアはこれと同じ意味。

```python
scores = torch.tensor([m @ x for m in memory])
```

この重みがmemoryの各データの重要度を表す。これで重み付き和をとる。

In [8]:
# 重みをかける
weighted_memory = torch.stack([w * m for w, m in zip(weights, memory)])
weighted_memory

tensor([[-0.1567,  0.3204,  1.0375, -0.5634,  1.3326],
        [-0.2532, -0.2796,  0.2979,  0.0152, -0.3389],
        [-0.0802,  0.1194, -0.0056,  0.1114, -0.0795]])

In [9]:
# 和をとる
y = weighted_memory.sum(dim=0)
y

tensor([-0.4900,  0.1601,  1.3299, -0.4369,  0.9143])

以下のようにまとめられる。

In [10]:
y = weights @ memory # 重み付き和
y

tensor([-0.4900,  0.1601,  1.3299, -0.4369,  0.9143])

以上がattention機構の演算の流れである。  
まとめるとこう。

In [14]:
scores = memory @ x # スコア (内積)
weights = F.softmax(scores, dim=-1) # 重み
y = memory.T @ weights # 重み付き和
y

tensor([-0.4900,  0.1601,  1.3299, -0.4369,  0.9143])

数式だとこうなる。

$$
\text{Attention}(\boldsymbol x,M) = M^T\text{softmax}(M\boldsymbol x)
$$

- $\boldsymbol x\in\R^{d}$ : 入力
- $M\in\R^{n\times d}$ : memory

### Seq2Seqへの導入

seq2seqのdecoderでは、このattentionを利用して、encoderが出力した全ての隠れ状態を参照する。  

現状、encoderの隠れ状態は最後のもののみがdecoderに渡っているが、可能なら全ての隠れ状態をdecoderに渡したい。しかし、encoderの隠れ状態の数はencoderへの入力の数によって変化し、可変長のデータをモデル内で扱うことは難しい。  
そこでattentionを用いる。decoderのある時間の隠れ状態を入力、encoderの全ての隠れ状態をmemoryとしてattentionを行う。こうすることで、encoderの隠れ状態から注目すべき重要な情報を都合よく抽出した固定長のベクトルを得ることが出来る。

In [11]:
seq_len = 3
hidden_size = 5
hs_enc = torch.randn(seq_len, hidden_size) # encoderが出力した全ての隠れ状態
h_dec = torch.randn(hidden_size) # ある時間tのdecoderの隠れ状態

scores = h_dec @ hs_enc.T # (seq_len,)
weights = F.softmax(scores, dim=-1)
y = weights @ hs_enc # (hidden_size,)
y # encoderの全ての隠れ状態の重要な部分に着目して得たベクトル

tensor([ 1.4763, -0.0759,  0.3263,  1.7235, -0.4055])

ちなみに、重みが正しく着目すべき点を表すかは、学習させてみないと分からない。  
この目的も、学習前の段階では期待に過ぎない。この仕組みを取り入れて学習させれば、次第に適切な重みが出力されるようになり、適切な出力が得られるようになるだろう。そうだといいな、ってだけ。

全ての隠れ状態を参照した固定長のベクトルを得るだけであれば、単に全ての隠れ状態を足すだけでもいい。ただ、重みを変えられるような枠組みを取り入れてあげれば学習が上手くいくんじゃね？ってだけ。そして本当にうまくいったからここで紹介されている。

また、重みを求める関数が内積でないといけない理由はない。2つのベクトルからスカラーを得る関数であれば何でもよい。  
内積は類似度を測ることができ、類似度が高いものに着目するという意味では適切に見えるが、そもそもデータ空間（？）が異なるので、それらの類似度は意味を持たない。  
重みを求める関数を内積として学習を進めれば、重要度が高くなるべきタイミングでその2つのベクトルが類似するように学習される、というだけ。

ただ実際はほとんどの場合で内積が使われる。それは内積という計算がシンプルだからってだけ。

#### Attention層

decoderの中の、attentionによって都合のいい隠れ状態を出力する部分は1つの層として見られる。  
複数時間の入力を考慮して以下のように表す。

$$
\text{Attention}(X,M) = \text{softmax}(XM^T)M
$$

- $X\in\R^{n_i\times d}$ : 入力
- $M\in\R^{n_m\times d}$ : memory

実装してみよう。

In [13]:
class Attention(nn.Module):
    def forward(self, x, hs):
        """
        x: (batch_size, seq_len_dec, hidden_size)
        hs: (batch_size, seq_len_enc, hidden_size)
        """
        scores = x @ hs.mT # (batch_size, seq_len_dec, seq_len_enc)
        weights = F.softmax(scores, dim=-1)
        h = weights @ hs # (batch_size, seq_len_dec, hidden_size)
        return h

In [14]:
batch_size, seq_len_dec, seq_len_enc, hidden_size = 2, 3, 4, 5
x = torch.randn(batch_size, seq_len_dec, hidden_size)
hs = torch.randn(batch_size, seq_len_enc, hidden_size)

attention = Attention()
h = attention(x, hs)
h.shape

torch.Size([2, 3, 5])


---

## MASK

padトークンがattentionの計算に含まれてしまうことを回避する。  
maskをかけてpadトークンに対応する重みが0になるようにする。

スコアに対して、対応する位置の値を$-\infty$にする。そうすればsoftmaxを計算したときにその部分が0になる。

$$
\mathrm{Attention}(\boldsymbol x,M) = \mathrm{softmax}(\boldsymbol xM^T-\infty) M
$$

こんなスコアがあったとする。

In [15]:
scores = torch.randn(5)
scores

tensor([-0.7732,  1.5053, -0.1447, -1.7741, -1.2302])

後ろの2つがpadトークンだったとすると、こんな感じでmaskをかけてやればいい。

In [16]:
mask = [False, False, False, True, True]
scores[mask] = -torch.inf
scores

tensor([-0.7732,  1.5053, -0.1447,    -inf,    -inf])

こう書いてもいい。

In [17]:
scores = torch.randn(5)
mask = torch.tensor([0, 0, 0, 1, 1])
scores.masked_fill_(mask, -torch.inf)
scores

tensor([-0.2346, -0.7934,  1.1168,    -inf,    -inf])

後はこれをsoftmaxに通す。

In [18]:
weights = F.softmax(scores, dim=-1)
weights

tensor([0.1840, 0.1052, 0.7108, 0.0000, 0.0000])

できた。これでpadトークンが無視されるようになる。

層としても実装する。

In [19]:
class Attention(nn.Module):
    def forward(self, x, hs, mask=None):
        """
        x: (batch_size, seq_len_dec, hidden_size)
        hs: (batch_size, seq_len_enc, hidden_size)
        mask: (batch_size, seq_len_enc), bool, padトークンの位置
        """
        scores = x @ hs.mT # (batch_size, seq_len_dec, seq_len_enc)
        if mask is not None:
            scores.masked_fill_(mask.unsqueeze(1), -torch.inf) # maskを適用
        weights = F.softmax(scores, dim=-1)
        h = weights @ hs # (batch_size, seq_len_dec, hidden_size)
        return h


---

## Attentionを用いたSeq2Seq

seq2seqにattention層を取り入れて翻訳モデルを作ってみよう。

### Encoder

In [20]:
class Encoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.RNN(embed_size, hidden_size, batch_first=True)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_positions = x == eos_id # (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, _ = self.lstm(x) # (batch_size, seq_len, hidden_size)
        h = hs[eos_positions].unsqueeze(0) # (1, batch_size, hidden_size)
        return hs, h

### Decoder

前章のものにattention層を追加する。attention層の前後は残差結合で繋ぐ。

In [21]:
class Decoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.attention = Attention()
        self.fc = nn.Linear(hidden_size * 2, n_vocab)

    def forward(self, x, hs_enc, hc, mask=None):
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, hc = self.lstm(x, hc)
            # hs: (batch_size, seq_len, hidden_size)
            # h: (1, batch_size, hidden_size)
            # c: (1, batch_size, hidden_size)
        z = self.attention(hs, hs_enc, mask) # (batch_size, seq_len, hidden_size)
        z = torch.cat([z, hs], dim=-1) # skip connection
            # (batch_size, seq_len, hidden_size * 2)
        y = self.fc(z) # (batch_size, seq_len, n_vocab)
        return y, hc

### Seq2Seq

全ての隠れ状態とpadトークンの位置をdecoderに渡すようにする。

In [22]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        hs, h = self.encoder(x_enc)
        mask = x_enc == pad_id
        hc = (h, torch.zeros_like(h))
        y, _ = self.decoder(x_dec, hs, hc, mask)
        return y

### 学習

In [23]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

def eval_model(model):
    model.eval()
    losses = []
    with torch.no_grad():
        for x_enc, x_dec, y_dec in test_loader:
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)
            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            losses.append(loss.item())
    loss = sum(losses) / len(losses)
    return loss

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_loss = eval_model(model)
            prog.memo(f'test: {test_loss:.5f}', no_step=True)
        prog.memo()

In [24]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [25]:
train(model, optimizer, n_epochs=200, prog_unit=20)

   1-20/200: ############################## 100% [00:02:18.57] loss train: 4.39482, test: 5.05719 
  21-40/200: ############################## 100% [00:02:16.78] loss train: 2.38485, test: 5.83749 
  41-60/200: ############################## 100% [00:02:20.04] loss train: 1.15364, test: 6.90943 
  61-80/200: ############################## 100% [00:02:19.90] loss train: 0.48066, test: 8.01863 
 81-100/200: ############################## 100% [00:02:18.72] loss train: 0.17752, test: 9.01301 
101-120/200: ############################## 100% [00:02:18.66] loss train: 0.07526, test: 9.78767 
121-140/200: ############################## 100% [00:02:18.80] loss train: 0.04308, test: 10.07249 
141-160/200: ############################## 100% [00:02:15.35] loss train: 0.03062, test: 10.54114 
161-180/200: ############################## 100% [00:02:14.11] loss train: 0.02107, test: 10.89103 
181-200/200: ############################## 100% [00:02:13.86] loss train: 0.01707, test: 11.14737 


通常のseq2seqよりも訓練データの誤差が小さくなった。

### 翻訳

In [26]:
def token_sampling(y, decisive=True):
    y.squeeze_(0)
    if decisive:
        token = y.argmax().item()
    else:
        y[unk_id] = -torch.inf
        probs = F.softmax(y, dim=-1)
        token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str,
    max_len: int = 100,
    decisive: bool = True,
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor([in_ids + [eos_id]], device=device)

    hs, h = model.encoder(in_ids)
    hc = (h, torch.zeros_like(h))
    next_token = bos_id

    token_ids = []
    while len(token_ids) < max_len and next_token != eos_id:
        x = torch.tensor([[next_token]], device=device)
        y, hc = model.decoder(x, hs, hc)
        next_token = token_sampling(y, decisive)
        token_ids.append(next_token)

    sentence = sp_en.decode(token_ids)
    return sentence

訓練データ

In [27]:
n = 5
for _ in range(n):
    i = random.randint(0, len(train_dataset))
    x, _, t = train_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: 彼にセンスのない詩を書いたり セーターを編んだりしました
output: And I was writing terrible poetry and knitting sweaters for him.
answer: And I was writing terrible poetry and knitting sweaters for him.

input: その後 2分間に 3人が2才のワン・ユーの側を通り過ぎます
output: Within two minutes, three people pass two-year-old Wang Yue by.
answer: Within two minutes, three people pass two-year-old Wang Yue by.

input: もう1つはNoksha-Yug Acceesで、 農村地域の物流を統合することを目的に マイクロファイナンスに基づく自助組織財団から基金を受けました。
output: And the other is Moksha-Yug Access, which is integrating rural supply chain on the foundations of self-help group-based microfinance.
answer: And the other is Moksha-Yug Access, which is integrating rural supply chain on the foundations of self-help group-based microfinance.

input: この小さな機器で何でもします
output: I do so many things on this little device.
answer: I do so many things on this little device.

input: 我々は暴力には力で対峙し 混沌には混沌で対峙しました
output: We met violence with force and chaos with chaos.
answer: We met violence with force and chaos with c

テストデータ

In [28]:
# test data
for _ in range(n):
    i = random.randint(0, len(test_dataset))
    x, _, t = test_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: しかし 私の実務経験から言うと 民間団体がこれを正しく実行し 他の団体と一緒に行動すると 特に政府や 国際機関 それらは大きな国際的団体と協力し それらの団体が 社会的責任を果たせるようになるのです その時 この魔法の三角形 すなわち民間団体と 政府と企業による三角形が より良い世界を作るために 私達すべてに大きなチャンスをもたらすのです
output: And if, there was a look, there's a ly Cord and alle that we had inside.
answer: But what I'm saying from my very practical experience: If civil society does it right and joins the other actors -- in particular, governments, governments and their international institutions, but also large international actors, in particular those which have committed themselves to corporate social responsibility -- then in this magical triangle between civil society, government and private sector, there is a tremendous chance for all of us to create a better world.

input: 地球全体としての生命を考えてみましょう ある意味 地球全体で生命ですし
output: It's been more or than a third system that have been a part of our species.
answer: Let's think of life as that entire planet because, in a sense, it is.

input: この会社のドアをくぐったとき 私はついにカミングアウトする―
output: He's a

In [29]:
# original
sentences = [
    'ありがとう。',
    '猫はかわいいね。',
    '上手く文章が書けるようになりました。'
]

for sentence in sentences:
    print('input:', sentence)
    print('output:', translate(model, sentence))
    print()

input: ありがとう。
output: Thank you.

input: 猫はかわいいね。
output: I'm here to do a magic.

input: 上手く文章が書けるようになりました。
output: I wanted to race that. I wanted to look at this time, and I was now going to make it all the first time in the  ⁇ 



びみょう。


---

## Attentionの可視化

